In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
from glob import glob
import json
import pathlib

import plotly.graph_objects as go
import numpy as np
from traitlets import dlink
import ipywidgets as widgets
from IPython.display import display, Image
from io import StringIO

from ipyfilechooser import FileChooser

from skimage.measure import label, regionprops, regionprops_table

from aai.io import dm3_lib as dm3
from aai.io import tem
from aai.image_segment import binary_segment
from aai.io import dm3_lib as dm3
from aai.io import tem
from aai.image_segment import binary_segment

In [ ]:
Image('dashboard.png', width=256, height=256)

# TEM Image Segmentation and Analysis
This notebook shows how to upload `.dm3` files of TEM images and analyze them.

- [1 Load data](#1-Load-data)
- [2 Analysis](#2-Analysis)
- [3 Modeling](#3-Modeling)
- [4 Results](#4-Results)

## 1 Load data

In [ ]:
fc = FileChooser()
fc.use_dir_icons = True
out_pl = widgets.Output()
lbl_img = widgets.Label()

plt.ioff()
fig = plt.figure()
fig.canvas.header_visible = False

In [ ]:
def on_data_change(change):
    lbl_img.value = ''
    file_path = fc.selected
    data = tem.TEMDataset(file_names=[file_path])
    img_data, metadata = data[0]
    
    display(plt.imshow(np.asarray(img_data)))
    plt.show()
    lbl_img.value = f'File: {metadata["name"]}; Units: {metadata["units"]}'
    fig.canvas.draw()
    fig.canvas.flush_events()

In [ ]:
fc.register_callback(on_data_change)

In [ ]:
widgets.AppLayout(header=fc, center=fig.canvas, footer=lbl_img)

## 2 Analysis

In [ ]:
button = widgets.Button(description="Segment Image")
output = widgets.Output()
display(widgets.VBox([button, output]))

def on_button_clicked(b):
    with output:
        if fc.selected is not None:
            file_path = fc.selected
            data = tem.TEMDataset(file_names=[file_path])
            image, metadata = data[0]
            regionlist, label_image, im_overlay = binary_segment(((image - np.amin(image))/(np.amax(image)-np.amin(image))), 0.01)
            labelmask = np.zeros(label_image.shape)
            labelmask[label_image > 0] = 1

            arealist = []
            for ri, region in enumerate(regionlist):

                if region.area < 500:
                    arealist.append(region.area)

            saveprops = ['area', 'bbox','centroid', 'coords', 'equivalent_diameter', 'label', 'perimeter']
            regiontable = regionprops_table(label_image, properties = saveprops)
            plt.imshow(im_overlay)
            plt.show()
        
            hist_fig = go.Figure(data=[go.Histogram(x=arealist)])
            hist_fig.update_layout(xaxis_title="Particle size (%s)" % metadata["units"],
                                  yaxis_title="Count")
            hist_fig.show()
        else:
            raise ValueError("Must upload data before segmentation!")

button.on_click(on_button_clicked)

## 3 Modeling

## 4 Results